In [17]:
# Importación de librerías esenciales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_squared_error, r2_score,
    accuracy_score, classification_report, confusion_matrix,
    precision_score, recall_score, f1_score,
    silhouette_score
)

# Configuración opcional para ignorar warnings y mejorar la visualización
import warnings
warnings.filterwarnings('ignore')


## --- 1. Carga y Preprocesamiento de Datos ---



# Ruta del archivo CSV dentro del entorno de Colab
file_path = '/content/laptop_price.csv'

try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='latin1')


# Ver las primeras filas
print(df.head())

   laptop_ID Company      Product   TypeName  Inches  \
0          1   Apple  MacBook Pro  Ultrabook    13.3   
1          2   Apple  Macbook Air  Ultrabook    13.3   
2          3      HP       250 G6   Notebook    15.6   
3          4   Apple  MacBook Pro  Ultrabook    15.4   
4          5   Apple  MacBook Pro  Ultrabook    13.3   

                     ScreenResolution                         Cpu   Ram  \
0  IPS Panel Retina Display 2560x1600        Intel Core i5 2.3GHz   8GB   
1                            1440x900        Intel Core i5 1.8GHz   8GB   
2                   Full HD 1920x1080  Intel Core i5 7200U 2.5GHz   8GB   
3  IPS Panel Retina Display 2880x1800        Intel Core i7 2.7GHz  16GB   
4  IPS Panel Retina Display 2560x1600        Intel Core i5 3.1GHz   8GB   

                Memory                           Gpu  OpSys  Weight  \
0            128GB SSD  Intel Iris Plus Graphics 640  macOS  1.37kg   
1  128GB Flash Storage        Intel HD Graphics 6000  macOS  1.34kg   

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# Convertir Ram de "8GB" → 8
df['Ram'] = df['Ram'].str.replace('GB', '').astype(int)

# Convertir Weight de "2.1kg" → 2.1
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)


Crear columna "demanda"

In [20]:
# Regla sencilla: precios bajos y más RAM → más demanda
np.random.seed(42)  # Para resultados reproducibles

base = 1000 - df['Price_euros'] * 0.7 + df['Ram'] * 5
ruido = np.random.normal(0, 30, size=len(df))  # un poco de aleatoriedad
df['Demanda'] = np.clip(base + ruido, 0, None)  # evitar negativos

# Verificar
print(df[['Price_euros', 'Ram', 'Demanda']].head())


   Price_euros  Ram     Demanda
0      1339.69    8  117.118425
1       898.94    8  406.594071
2       575.00    8  656.930656
3      2537.45   16    0.000000
4      1803.60    8    0.000000


In [21]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# 1. Seleccionamos las columnas de entrada
features = ['Company', 'TypeName', 'Ram', 'Weight', 'Price_euros']
X_raw = df[features]

# 2. One-hot encoding para las categóricas
categorical_cols = ['Company', 'TypeName']
numerical_cols = ['Ram', 'Weight', 'Price_euros']

# One-hot encoding (devuelve un array)
encoder = OneHotEncoder(sparse_output=False)
X_cat = encoder.fit_transform(X_raw[categorical_cols])

# Escalamiento de las numéricas (media = 0, varianza = 1)
scaler = StandardScaler()
X_num = scaler.fit_transform(X_raw[numerical_cols])

# 3. Concatenamos todo
import numpy as np
X = np.hstack([X_cat, X_num])

# 4. Salida: y = Demanda
y = df['Demanda'].values.reshape(-1, 1)

# 5. Confirmamos formas
print("Forma de X:", X.shape)
print("Forma de y:", y.shape)


Forma de X: (1303, 28)
Forma de y: (1303, 1)


In [22]:

# Funciones de activación
def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
    s = sigmoid(x)
    return s * (1 - s)

def tanh(x):
    return np.tanh(x)

def tanh_deriv(x):
    return 1 - np.tanh(x) ** 2

# Diccionario de activaciones
activations = {
    'relu': (relu, relu_deriv),
    'sigmoid': (sigmoid, sigmoid_deriv),
    'tanh': (tanh, tanh_deriv)
}

class NeuralNetwork:
    def __init__(self, layers, activation='relu'):
        self.layers = layers
        self.activation_name = activation
        self.activation, self.activation_deriv = activations[activation]
        self._init_weights()

    def _init_weights(self):
        self.weights = []
        self.biases = []
        for i in range(len(self.layers) - 1):
            weight = np.random.randn(self.layers[i], self.layers[i+1]) * np.sqrt(2. / self.layers[i])
            bias = np.zeros((1, self.layers[i+1]))
            self.weights.append(weight)
            self.biases.append(bias)

    def forward(self, X):
        self.a = [X]
        self.z = []
        for i in range(len(self.weights) - 1):
            z = self.a[-1] @ self.weights[i] + self.biases[i]
            self.z.append(z)
            self.a.append(self.activation(z))
        # Capa de salida: sin activación (regresión)
        z = self.a[-1] @ self.weights[-1] + self.biases[-1]
        self.z.append(z)
        self.a.append(z)
        return z

    def backward(self, y, learning_rate):
        m = y.shape[0]
        delta = (self.a[-1] - y) / m  # derivada MSE
        for i in reversed(range(len(self.weights))):
            dw = self.a[i].T @ delta
            db = np.sum(delta, axis=0, keepdims=True)
            self.weights[i] -= learning_rate * dw
            self.biases[i] -= learning_rate * db
            if i != 0:
                delta = (delta @ self.weights[i].T) * self.activation_deriv(self.z[i-1])

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(y, learning_rate)
            if epoch % 100 == 0:
                loss = np.mean((output - y) ** 2)
                print(f"Epoch {epoch} | Loss: {loss:.4f}")

    def predict(self, X):
        return self.forward(X)


In [23]:
import sys
sys.path.append('./src')


In [25]:
# 1. Validar datos
assert not np.any(np.isnan(X))
assert not np.any(np.isnan(y))

# 2. Escalar X e Y
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_Y = MinMaxScaler()
y_scaled = scaler_Y.fit_transform(y)

# 3. Entrenar con tasa baja y otra activación si puedes
nn = NeuralNetwork(layers=[X.shape[1], 64, 32, 1], activation='tanh')
nn.train(X_scaled, y_scaled, epochs=1000, learning_rate=0.0001)

Epoch 0 | Loss: 0.4828
Epoch 100 | Loss: 0.4536
Epoch 200 | Loss: 0.4269
Epoch 300 | Loss: 0.4023
Epoch 400 | Loss: 0.3796
Epoch 500 | Loss: 0.3588
Epoch 600 | Loss: 0.3396
Epoch 700 | Loss: 0.3219
Epoch 800 | Loss: 0.3055
Epoch 900 | Loss: 0.2904


In [26]:


# Suponiendo que ya tienes X e y listos
nn = NeuralNetwork(layers=[X.shape[1], 64, 32, 1], activation='relu')
nn.train(X_scaled, y, epochs=1000, learning_rate=0.0001)

# Predecir sobre el mismo conjunto
predictions = nn.predict(X)


Epoch 0 | Loss: 207913.1992
Epoch 100 | Loss: 1386.0208
Epoch 200 | Loss: 752.6918
Epoch 300 | Loss: 675.2305
Epoch 400 | Loss: 641.8407
Epoch 500 | Loss: 622.9860
Epoch 600 | Loss: 609.6185
Epoch 700 | Loss: 599.6755
Epoch 800 | Loss: 592.6674
Epoch 900 | Loss: 587.1037


In [27]:
nn = NeuralNetwork(layers=[X.shape[1], 128, 64, 32, 1], activation='tanh')

predictions = nn.predict(X_scaled)
predictions_original = scaler_Y.inverse_transform(predictions)

In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # X debe ser solo valores numéricos

# Luego pasa esto a la red:
nn.train(X_scaled, y, epochs=1000, learning_rate=0.01)


Epoch 0 | Loss: 208018.9045
Epoch 100 | Loss: 52535.3969
Epoch 200 | Loss: 43561.0056
Epoch 300 | Loss: 41877.0724
Epoch 400 | Loss: 41581.9396
Epoch 500 | Loss: 41521.6333
Epoch 600 | Loss: 41506.7951
Epoch 700 | Loss: 41487.1184
Epoch 800 | Loss: 41482.2995
Epoch 900 | Loss: 41478.0002


In [29]:
nn.train(X_scaled, y, epochs=1000, learning_rate=0.001)


Epoch 0 | Loss: 41473.9642
Epoch 100 | Loss: 41473.5719
Epoch 200 | Loss: 41473.1811
Epoch 300 | Loss: 41472.7916
Epoch 400 | Loss: 41472.4023
Epoch 500 | Loss: 41472.0108
Epoch 600 | Loss: 41471.6036
Epoch 700 | Loss: 41466.5840
Epoch 800 | Loss: 41459.5004
Epoch 900 | Loss: 41458.7325


In [30]:
def train(self, X, y, epochs, learning_rate):
    for epoch in range(epochs):
        output = self.forward(X)
        self.backward(y, learning_rate)
        if epoch % 100 == 0:
            loss = np.mean((output - y) ** 2)
            print(f"Epoch {epoch} | Loss: {loss:.4f} | Output Sample: {output[0][:5]}")


## Revisión y mejora de la clase `neuralnetwork`


Asegurar que la inicialización de pesos implemente Xavier o He, dependiendo de la función de activación. Verificar que el forward y backward propagation estén correctamente implementados para múltiples capas ocultas. Asegurar que la capa de salida no tenga función de activación para un problema de regresión. Añadir el cálculo de la pérdida (MSE) en cada época de entrenamiento y mostrarla periódicamente. Incluir la opción de elegir la función de activación para las capas ocultas.


In [31]:
class NeuralNetwork:
    def __init__(self, layers, activation='relu'):
        self.layers = layers
        self.activation_name = activation
        self.activation, self.activation_deriv = activations[activation]
        self._init_weights()

    def _init_weights(self):
        self.weights = []
        self.biases = []
        for i in range(len(self.layers) - 1):
            # Inicialización de pesos
            input_dim = self.layers[i]
            output_dim = self.layers[i+1]
            if self.activation_name == 'relu':
                # Inicialización de He para ReLU
                weight = np.random.randn(input_dim, output_dim) * np.sqrt(2. / input_dim)
            else:
                # Inicialización de Xavier para Sigmoid y Tanh
                weight = np.random.randn(input_dim, output_dim) * np.sqrt(1. / input_dim)

            bias = np.zeros((1, output_dim))
            self.weights.append(weight)
            self.biases.append(bias)

    def forward(self, X):
        self.a = [X]
        self.z = []
        # Propagación a través de las capas ocultas
        for i in range(len(self.weights) - 1):
            z = self.a[-1] @ self.weights[i] + self.biases[i]
            self.z.append(z)
            self.a.append(self.activation(z))
        # Capa de salida: sin activación (regresión)
        z = self.a[-1] @ self.weights[-1] + self.biases[-1]
        self.z.append(z)
        self.a.append(z)
        return z

    def backward(self, y, learning_rate):
        m = y.shape[0]
        # Derivada de la pérdida (MSE) para la capa de salida
        delta = (self.a[-1] - y) / m

        # Retropropagación a través de las capas
        for i in reversed(range(len(self.weights))):
            dw = self.a[i].T @ delta
            db = np.sum(delta, axis=0, keepdims=True)

            self.weights[i] -= learning_rate * dw
            self.biases[i] -= learning_rate * db

            if i != 0:
                # Propagación del error a la capa anterior
                delta = (delta @ self.weights[i].T) * self.activation_deriv(self.z[i-1])

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(y, learning_rate)

            # Cálculo de la pérdida (MSE)
            loss = np.mean((output - y) ** 2)

            if epoch % 100 == 0:
                print(f"Epoch {epoch} | Loss: {loss:.4f}")

    def predict(self, X):
        return self.forward(X)

## Preparación de los datos

Confirmar que los datos de entrada (`X`) y salida (`y`) están correctamente preparados, incluyendo el one-hot encoding para variables categóricas y el escalado para variables numéricas. Escalar la variable objetivo (`y`) si es necesario para mejorar la estabilidad del entrenamiento, y recordar desescalar las predicciones al final.


In [32]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
import numpy as np

# 1. Seleccionamos las columnas de entrada
features = ['Company', 'TypeName', 'Ram', 'Weight', 'Price_euros']
X_raw = df[features]

# 2. One-hot encoding para las categóricas
categorical_cols = ['Company', 'TypeName']
numerical_cols = ['Ram', 'Weight', 'Price_euros']

# One-hot encoding (devuelve un array)
encoder = OneHotEncoder(sparse_output=False)
X_cat = encoder.fit_transform(X_raw[categorical_cols])

# Escalamiento de las numéricas (media = 0, varianza = 1)
scaler = StandardScaler()
X_num = scaler.fit_transform(X_raw[numerical_cols])

# 3. Concatenamos todo
X = np.hstack([X_cat, X_num])

# 4. Salida: y = Demanda
y = df['Demanda'].values.reshape(-1, 1)

# 5. Escalar la variable objetivo y
scaler_Y = MinMaxScaler()
y_scaled = scaler_Y.fit_transform(y)

# 6. Confirmamos formas
print("Forma de X:", X.shape)
print("Forma de y (original):", y.shape)
print("Forma de y (escalada):", y_scaled.shape)

# Verificar si hay NaNs o infs después de las transformaciones
print("NaNs en X:", np.any(np.isnan(X)))
print("Infs en X:", np.any(np.isinf(X)))
print("NaNs en y_scaled:", np.any(np.isnan(y_scaled)))
print("Infs en y_scaled:", np.any(np.isinf(y_scaled)))

Forma de X: (1303, 28)
Forma de y (original): (1303, 1)
Forma de y (escalada): (1303, 1)
NaNs en X: False
Infs en X: False
NaNs en y_scaled: False
Infs en y_scaled: False


In [33]:

try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='latin1')

# Convertir Ram de "8GB" → 8
df['Ram'] = df['Ram'].str.replace('GB', '').astype(int)

# Convertir Weight de "2.1kg" → 2.1
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)

# Recreate the 'Demanda' column
np.random.seed(42)
base = 1000 - df['Price_euros'] * 0.7 + df['Ram'] * 5
ruido = np.random.normal(0, 30, size=len(df))
df['Demanda'] = np.clip(base + ruido, 0, None)

# Now proceed with the data preparation steps
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

# 1. Seleccionamos las columnas de entrada
features = ['Company', 'TypeName', 'Ram', 'Weight', 'Price_euros']
X_raw = df[features]

# 2. One-hot encoding para las categóricas
categorical_cols = ['Company', 'TypeName']
numerical_cols = ['Ram', 'Weight', 'Price_euros']

# One-hot encoding (devuelve un array)
encoder = OneHotEncoder(sparse_output=False)
X_cat = encoder.fit_transform(X_raw[categorical_cols])

# Escalamiento de las numéricas (media = 0, varianza = 1)
scaler = StandardScaler()
X_num = scaler.fit_transform(X_raw[numerical_cols])

# 3. Concatenamos todo
X = np.hstack([X_cat, X_num])

# 4. Salida: y = Demanda
y = df['Demanda'].values.reshape(-1, 1)

# 5. Escalar la variable objetivo y
scaler_Y = MinMaxScaler()
y_scaled = scaler_Y.fit_transform(y)

# 6. Confirmamos formas
print("Forma de X:", X.shape)
print("Forma de y (original):", y.shape)
print("Forma de y (escalada):", y_scaled.shape)

# Verificar si hay NaNs o infs después de las transformaciones
print("NaNs en X:", np.any(np.isnan(X)))
print("Infs en X:", np.any(np.isinf(X)))
print("NaNs en y_scaled:", np.any(np.isnan(y_scaled)))
print("Infs en y_scaled:", np.any(np.isinf(y_scaled)))

Forma de X: (1303, 28)
Forma de y (original): (1303, 1)
Forma de y (escalada): (1303, 1)
NaNs en X: False
Infs en X: False
NaNs en y_scaled: False
Infs en y_scaled: False


In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler


try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='latin1')

# Convertir Ram de "8GB" → 8
df['Ram'] = df['Ram'].str.replace('GB', '').astype(int)

# Convertir Weight de "2.1kg" → 2.1
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)

# Recreate the 'Demanda' column
np.random.seed(42)
base = 1000 - df['Price_euros'] * 0.7 + df['Ram'] * 5
ruido = np.random.normal(0, 30, size=len(df))
df['Demanda'] = np.clip(base + ruido, 0, None)

# Now proceed with the data preparation steps

# 1. Seleccionamos las columnas de entrada
features = ['Company', 'TypeName', 'Ram', 'Weight', 'Price_euros']
X_raw = df[features]

# 2. One-hot encoding para las categóricas
categorical_cols = ['Company', 'TypeName']
numerical_cols = ['Ram', 'Weight', 'Price_euros']

# One-hot encoding (devuelve un array)
encoder = OneHotEncoder(sparse_output=False)
X_cat = encoder.fit_transform(X_raw[categorical_cols])

# Escalamiento de las numéricas (media = 0, varianza = 1)
scaler = StandardScaler()
X_num = scaler.fit_transform(X_raw[numerical_cols])

# 3. Concatenamos todo
X = np.hstack([X_cat, X_num])

# 4. Salida: y = Demanda
y = df['Demanda'].values.reshape(-1, 1)

# 5. Escalar la variable objetivo y
scaler_Y = MinMaxScaler()
y_scaled = scaler_Y.fit_transform(y)

# 6. Confirmamos formas
print("Forma de X:", X.shape)
print("Forma de y (original):", y.shape)
print("Forma de y (escalada):", y_scaled.shape)

# Verificar si hay NaNs o infs después de las transformaciones
print("NaNs en X:", np.any(np.isnan(X)))
print("Infs en X:", np.any(np.isinf(X)))
print("NaNs en y_scaled:", np.any(np.isnan(y_scaled)))
print("Infs en y_scaled:", np.any(np.isinf(y_scaled)))

Forma de X: (1303, 28)
Forma de y (original): (1303, 1)
Forma de y (escalada): (1303, 1)
NaNs en X: False
Infs en X: False
NaNs en y_scaled: False
Infs en y_scaled: False


In [35]:

try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='latin1')

# Convertir Ram de "8GB" → 8
df['Ram'] = df['Ram'].str.replace('GB', '').astype(int)

# Convertir Weight de "2.1kg" → 2.1
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)

# Recreate the 'Demanda' column
np.random.seed(42)
base = 1000 - df['Price_euros'] * 0.7 + df['Ram'] * 5
ruido = np.random.normal(0, 30, size=len(df))
df['Demanda'] = np.clip(base + ruido, 0, None)

# Now proceed with the data preparation steps

# 1. Seleccionamos las columnas de entrada
features = ['Company', 'TypeName', 'Ram', 'Weight', 'Price_euros']
X_raw = df[features]

# 2. One-hot encoding para las categóricas
categorical_cols = ['Company', 'TypeName']
numerical_cols = ['Ram', 'Weight', 'Price_euros']

# One-hot encoding (devuelve un array)
encoder = OneHotEncoder(sparse_output=False)
X_cat = encoder.fit_transform(X_raw[categorical_cols])

# Escalamiento de las numéricas (media = 0, varianza = 1)
scaler = StandardScaler()
X_num = scaler.fit_transform(X_raw[numerical_cols])

# 3. Concatenamos todo
X = np.hstack([X_cat, X_num])

# 4. Salida: y = Demanda
y = df['Demanda'].values.reshape(-1, 1)

# 5. Escalar la variable objetivo y
scaler_Y = MinMaxScaler()
y_scaled = scaler_Y.fit_transform(y)

# 6. Confirmamos formas
print("Forma de X:", X.shape)
print("Forma de y (original):", y.shape)
print("Forma de y (escalada):", y_scaled.shape)

# Verificar si hay NaNs o infs después de las transformaciones
print("NaNs en X:", np.any(np.isnan(X)))
print("Infs en X:", np.any(np.isinf(X)))
print("NaNs en y_scaled:", np.any(np.isnan(y_scaled)))
print("Infs en y_scaled:", np.any(np.isinf(y_scaled)))

Forma de X: (1303, 28)
Forma de y (original): (1303, 1)
Forma de y (escalada): (1303, 1)
NaNs en X: False
Infs en X: False
NaNs en y_scaled: False
Infs en y_scaled: False


## División de datos

Dividir el conjunto de datos en entrenamiento y prueba para evaluar el rendimiento del modelo en datos no vistos.


In [36]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Verificar las formas de los conjuntos resultantes
print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)

Forma de X_train: (1042, 28)
Forma de X_test: (261, 28)
Forma de y_train: (1042, 1)
Forma de y_test: (261, 1)


In [37]:

try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='latin1')

# Convertir Ram de "8GB" → 8
df['Ram'] = df['Ram'].str.replace('GB', '').astype(int)

# Convertir Weight de "2.1kg" → 2.1
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)

# Recreate the 'Demanda' column
np.random.seed(42)
base = 1000 - df['Price_euros'] * 0.7 + df['Ram'] * 5
ruido = np.random.normal(0, 30, size=len(df))
df['Demanda'] = np.clip(base + ruido, 0, None)

# Now proceed with the data preparation steps

# 1. Seleccionamos las columnas de entrada
features = ['Company', 'TypeName', 'Ram', 'Weight', 'Price_euros']
X_raw = df[features]

# 2. One-hot encoding para las categóricas
categorical_cols = ['Company', 'TypeName']
numerical_cols = ['Ram', 'Weight', 'Price_euros']

# One-hot encoding (devuelve un array)
encoder = OneHotEncoder(sparse_output=False)
X_cat = encoder.fit_transform(X_raw[categorical_cols])

# Escalamiento de las numéricas (media = 0, varianza = 1)
scaler = StandardScaler()
X_num = scaler.fit_transform(X_raw[numerical_cols])

# 3. Concatenamos todo
X = np.hstack([X_cat, X_num])

# 4. Salida: y = Demanda
y = df['Demanda'].values.reshape(-1, 1)

# 5. Escalar la variable objetivo y
scaler_Y = MinMaxScaler()
y_scaled = scaler_Y.fit_transform(y)

# 6. Verificar si hay NaNs o infs after transformations
print("NaNs en X:", np.any(np.isnan(X)))
print("Infs en X:", np.any(np.isinf(X)))
print("NaNs en y_scaled:", np.any(np.isnan(y_scaled)))
print("Infs en y_scaled:", np.any(np.isinf(y_scaled)))

# Now split the data
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

# Verificar las formas de los conjuntos resultantes
print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)

NaNs en X: False
Infs en X: False
NaNs en y_scaled: False
Infs en y_scaled: False
Forma de X_train: (1042, 28)
Forma de X_test: (261, 28)
Forma de y_train: (1042, 1)
Forma de y_test: (261, 1)


In [38]:

try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='latin1')
except FileNotFoundError:
    print(f"Error: El archivo no se encontró en la ruta especificada: {file_path}")
    # Attempt to mount drive again just in case
    from google.colab import drive
    drive.mount('/content/drive')
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='latin1')
    except FileNotFoundError:
        print("Error: El archivo aún no se encontró después de intentar montar el Drive.")
        df = None # Set df to None to indicate failure

if df is not None:
    # Convertir Ram de "8GB" → 8
    df['Ram'] = df['Ram'].str.replace('GB', '').astype(int)

    # Convertir Weight de "2.1kg" → 2.1
    df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)

    # Recreate the 'Demanda' column
    np.random.seed(42)
    base = 1000 - df['Price_euros'] * 0.7 + df['Ram'] * 5
    ruido = np.random.normal(0, 30, size=len(df))
    df['Demanda'] = np.clip(base + ruido, 0, None)

    # Now proceed with the data preparation steps

    # 1. Seleccionamos las columnas de entrada
    features = ['Company', 'TypeName', 'Ram', 'Weight', 'Price_euros']
    X_raw = df[features]

    # 2. One-hot encoding para las categóricas
    categorical_cols = ['Company', 'TypeName']
    numerical_cols = ['Ram', 'Weight', 'Price_euros']

    # One-hot encoding (devuelve un array)
    encoder = OneHotEncoder(sparse_output=False)
    X_cat = encoder.fit_transform(X_raw[categorical_cols])

    # Escalamiento de las numéricas (media = 0, varianza = 1)
    scaler = StandardScaler()
    X_num = scaler.fit_transform(X_raw[numerical_cols])

    # 3. Concatenamos todo
    X = np.hstack([X_cat, X_num])

    # 4. Salida: y = Demanda
    y = df['Demanda'].values.reshape(-1, 1)

    # 5. Escalar la variable objetivo y
    scaler_Y = MinMaxScaler()
    y_scaled = scaler_Y.fit_transform(y)

    # 6. Verificar si hay NaNs o infs after transformations
    print("NaNs en X:", np.any(np.isnan(X)))
    print("Infs en X:", np.any(np.isinf(X)))
    print("NaNs en y_scaled:", np.any(np.isnan(y_scaled)))
    print("Infs en y_scaled:", np.any(np.isinf(y_scaled)))

    # Now split the data
    from sklearn.model_selection import train_test_split

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

    # Verificar las formas de los conjuntos resultantes
    print("Forma de X_train:", X_train.shape)
    print("Forma de X_test:", X_test.shape)
    print("Forma de y_train:", y_train.shape)
    print("Forma de y_test:", y_test.shape)
else:
    print("No se pudo cargar el DataFrame. La división de datos no se realizará.")


NaNs en X: False
Infs en X: False
NaNs en y_scaled: False
Infs en y_scaled: False
Forma de X_train: (1042, 28)
Forma de X_test: (261, 28)
Forma de y_train: (1042, 1)
Forma de y_test: (261, 1)


## Entrenamiento del modelo

### Subtask:
Instanciar la clase `NeuralNetwork` con una arquitectura inicial (al menos 2 capas ocultas) utilizando los datos escalados. Entrenar el modelo utilizando el conjunto de entrenamiento, ajustando los hiperparámetros como la tasa de aprendizaje y el número de épocas.


In [43]:
# Ruta del archivo CSV dentro del entorno de Colab
file_path = '/content/laptop_price.csv'

try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='latin1')
except FileNotFoundError:
    print(f"Error: El archivo no se encontró en la ruta especificada: {file_path}")
    # Attempt to mount drive again just in case
    from google.colab import drive
    drive.mount('/content/drive')
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='latin1')
    except FileNotFoundError:
        print("Error: El archivo aún no se encontró después de intentar montar el Drive.")
        df = None # Set df to None to indicate failure

if df is not None:
    # Convertir Ram de "8GB" → 8
    df['Ram'] = df['Ram'].str.replace('GB', '').astype(int)

    # Convertir Weight de "2.1kg" → 2.1
    df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)

    # Recreate the 'Demanda' column
    np.random.seed(42)
    base = 1000 - df['Price_euros'] * 0.7 + df['Ram'] * 5
    ruido = np.random.normal(0, 30, size=len(df))
    df['Demanda'] = np.clip(base + ruido, 0, None)

    # Now proceed with the data preparation steps

    # 1. Seleccionamos las columnas de entrada
    features = ['Company', 'TypeName', 'Ram', 'Weight', 'Price_euros']
    X_raw = df[features]

    # 2. One-hot encoding para las categóricas
    categorical_cols = ['Company', 'TypeName']
    numerical_cols = ['Ram', 'Weight', 'Price_euros']

    # One-hot encoding (devuelve un array)
    encoder = OneHotEncoder(sparse_output=False)
    X_cat = encoder.fit_transform(X_raw[categorical_cols])

    # Escalamiento de las numéricas (media = 0, varianza = 1)
    scaler = StandardScaler()
    X_num = scaler.fit_transform(X_raw[numerical_cols])

    # 3. Concatenamos todo
    X = np.hstack([X_cat, X_num])

    # 4. Salida: y = Demanda
    y = df['Demanda'].values.reshape(-1, 1)

    # 5. Escalar la variable objetivo y
    scaler_Y = MinMaxScaler()
    y_scaled = scaler_Y.fit_transform(y)

    # 6. Verificar si hay NaNs o infs after transformations
    print("NaNs en X:", np.any(np.isnan(X)))
    print("Infs en X:", np.any(np.isinf(X)))
    print("NaNs en y_scaled:", np.any(np.isnan(y_scaled)))
    print("Infs en y_scaled:", np.any(np.isinf(y_scaled)))

    # Now split the data
    from sklearn.model_selection import train_test_split

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

    # Verificar las formas de los conjuntos resultantes
    print("Forma de X_train:", X_train.shape)
    print("Forma de X_test:", X_test.shape)
    print("Forma de y_train:", y_train.shape)
    print("Forma de y_test:", y_test.shape)

    input_layer_size = X_train.shape[1]
    hidden_layer_1_size = 64
    hidden_layer_2_size = 32
    output_layer_size = 1 # For regression

    layers = [input_layer_size, hidden_layer_1_size, hidden_layer_2_size, output_layer_size]

    # Instantiate the Neural Network
    nn = NeuralNetwork(layers=layers, activation='tanh') # Using tanh as an initial activation

    # Train the model
    epochs = 1000
    learning_rate = 0.001 # Initial learning rate

    print("\nIniciando entrenamiento de la Red Neuronal...")
    nn.train(X_train, y_train, epochs=epochs, learning_rate=learning_rate)

else:
    print("No se pudo cargar el DataFrame. La división de datos y el entrenamiento no se realizarán.")

NaNs en X: False
Infs en X: False
NaNs en y_scaled: False
Infs en y_scaled: False
Forma de X_train: (1042, 28)
Forma de X_test: (261, 28)
Forma de y_train: (1042, 1)
Forma de y_test: (261, 1)

Iniciando entrenamiento de la Red Neuronal...
Epoch 0 | Loss: 0.2132
Epoch 100 | Loss: 0.1233
Epoch 200 | Loss: 0.0778
Epoch 300 | Loss: 0.0540
Epoch 400 | Loss: 0.0409
Epoch 500 | Loss: 0.0333
Epoch 600 | Loss: 0.0286
Epoch 700 | Loss: 0.0256
Epoch 800 | Loss: 0.0235
Epoch 900 | Loss: 0.0219



Realizar predicciones sobre el conjunto de prueba utilizando el modelo entrenado. Desescalar las predicciones si la variable objetivo fue escalada. Calcular métricas de evaluación para regresión (por ejemplo, MSE, RMSE, R²) para evaluar el rendimiento del modelo.


In [45]:

# Functions de activación
def relu(x):
    return np.maximum(0, x)

def relu_deriv(x):
    return (x > 0).astype(float)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
    s = sigmoid(x)
    return s * (1 - s)

def tanh(x):
    return np.tanh(x)

def tanh_deriv(x):
    return 1 - np.tanh(x) ** 2

# Diccionario de activaciones
activations = {
    'relu': (relu, relu_deriv),
    'sigmoid': (sigmoid, sigmoid_deriv),
    'tanh': (tanh, tanh_deriv)
}

class NeuralNetwork:
    def __init__(self, layers, activation='relu'):
        self.layers = layers
        self.activation_name = activation
        self.activation, self.activation_deriv = activations[activation]
        self._init_weights()

    def _init_weights(self):
        self.weights = []
        self.biases = []
        for i in range(len(self.layers) - 1):
            # Inicialización de pesos
            input_dim = self.layers[i]
            output_dim = self.layers[i+1]
            if self.activation_name == 'relu':
                # Inicialización de He para ReLU
                weight = np.random.randn(input_dim, output_dim) * np.sqrt(2. / input_dim)
            else:
                # Inicialización de Xavier para Sigmoid y Tanh
                weight = np.random.randn(input_dim, output_dim) * np.sqrt(1. / input_dim)

            bias = np.zeros((1, output_dim))
            self.weights.append(weight)
            self.biases.append(bias)

    def forward(self, X):
        self.a = [X]
        self.z = []
        # Propagación a través de las capas ocultas
        for i in range(len(self.weights) - 1):
            z = self.a[-1] @ self.weights[i] + self.biases[i]
            self.z.append(z)
            self.a.append(self.activation(z))
        # Capa de salida: sin activación (regresión)
        z = self.a[-1] @ self.weights[-1] + self.biases[-1]
        self.z.append(z)
        self.a.append(z)
        return z

    def backward(self, y, learning_rate):
        m = y.shape[0]
        # Derivada de la pérdida (MSE) para la capa de salida
        delta = (self.a[-1] - y) / m

        # Retropropagación a través de las capas
        for i in reversed(range(len(self.weights))):
            dw = self.a[i].T @ delta
            db = np.sum(delta, axis=0, keepdims=True)

            self.weights[i] -= learning_rate * dw
            self.biases[i] -= learning_rate * db

            if i != 0:
                # Propagación del error a la capa anterior
                delta = (delta @ self.weights[i].T) * self.activation_deriv(self.z[i-1])

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(y, learning_rate)

            # Cálculo de la pérdida (MSE)
            loss = np.mean((output - y) ** 2)

            if epoch % 100 == 0:
                print(f"Epoch {epoch} | Loss: {loss:.4f}")

    def predict(self, X):
        return self.forward(X)


# Re-load, process, and split the data
file_path = '/content/laptop_price.csv'

try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='latin1')
except FileNotFoundError:
    print(f"Error: El archivo no se encontró en la ruta especificada: {file_path}")
    # Attempt to mount drive again just in case
    from google.colab import drive
    drive.mount('/content/drive')
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='latin1')
    except FileNotFoundError:
        print("Error: El archivo aún no se encontró después de intentar montar el Drive.")
        df = None # Set df to None to indicate failure

if df is not None:
    # Convertir Ram de "8GB" → 8
    df['Ram'] = df['Ram'].str.replace('GB', '').astype(int)

    # Convertir Weight de "2.1kg" → 2.1
    df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)

    # Recreate the 'Demanda' column
    np.random.seed(42)
    base = 1000 - df['Price_euros'] * 0.7 + df['Ram'] * 5
    ruido = np.random.normal(0, 30, size=len(df))
    df['Demanda'] = np.clip(base + ruido, 0, None)

    # Now proceed with the data preparation steps

    # 1. Seleccionamos las columnas de entrada
    features = ['Company', 'TypeName', 'Ram', 'Weight', 'Price_euros']
    X_raw = df[features]

    # 2. One-hot encoding para las categóricas
    categorical_cols = ['Company', 'TypeName']
    numerical_cols = ['Ram', 'Weight', 'Price_euros']

    # One-hot encoding (devuelve un array)
    encoder = OneHotEncoder(sparse_output=False)
    X_cat = encoder.fit_transform(X_raw[categorical_cols])

    # Escalamiento de las numéricas (media = 0, varianza = 1)
    scaler = StandardScaler()
    X_num = scaler.fit_transform(X_raw[numerical_cols])

    # 3. Concatenamos todo
    X = np.hstack([X_cat, X_num])

    # 4. Salida: y = Demanda
    y = df['Demanda'].values.reshape(-1, 1)

    # 5. Escalar la variable objetivo y
    scaler_Y = MinMaxScaler()
    y_scaled = scaler_Y.fit_transform(y)

    # 6. Verificar si hay NaNs o infs after transformations
    print("NaNs en X:", np.any(np.isnan(X)))
    print("Infs en X:", np.any(np.isinf(X)))
    print("NaNs en y_scaled:", np.any(np.isnan(y_scaled)))
    print("Infs en y_scaled:", np.any(np.isinf(y_scaled)))

    # Now split the data
    from sklearn.model_selection import train_test_split

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

    # Verificar las formas de los conjuntos resultantes
    print("Forma de X_train:", X_train.shape)
    print("Forma de X_test:", X_test.shape)
    print("Forma de y_train:", y_train.shape)
    print("Forma de y_test:", y_test.shape)

    # Instantiate and train the Neural Network
    # Define the architecture
    input_layer_size = X_train.shape[1]
    hidden_layer_1_size = 64
    hidden_layer_2_size = 32
    output_layer_size = 1 # For regression

    layers = [input_layer_size, hidden_layer_1_size, hidden_layer_2_size, output_layer_size]

    # Instantiate the Neural Network
    nn = NeuralNetwork(layers=layers, activation='tanh') # Using tanh as an initial activation

    # Train the model
    epochs = 1000
    learning_rate = 0.001 # Initial learning rate

    print("\nIniciando entrenamiento de la Red Neuronal...")
    nn.train(X_train, y_train, epochs=epochs, learning_rate=learning_rate)

    # 3. Utiliza el modelo nn para hacer predicciones sobre el conjunto de prueba X_test
    print("\nRealizando predicciones en el conjunto de prueba...")
    predictions_scaled = nn.predict(X_test)

    # 4. Desescala las predicciones predictions_scaled
    predictions_original = scaler_Y.inverse_transform(predictions_scaled)

    # 5. Desescala el conjunto de prueba real y_test
    y_test_original = scaler_Y.inverse_transform(y_test)

    # 6. Calcula el Error Cuadrático Medio (MSE)
    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(y_test_original, predictions_original)

    # 7. Calcula la Raíz del Error Cuadrático Medio (RMSE)
    rmse = np.sqrt(mse)

    # 8. Calcula el coeficiente de determinación R²
    from sklearn.metrics import r2_score
    r2 = r2_score(y_test_original, predictions_original)

    # 9. Imprime los valores de MSE, RMSE y R²
    print("\nMétricas de Evaluación en el Conjunto de Prueba:")
    print(f"Error Cuadrático Medio (MSE): {mse:.4f}")
    print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.4f}")
    print(f"Coeficiente de Determinación (R²): {r2:.4f}")

else:
    print("No se pudo cargar el DataFrame. Las predicciones y la evaluación no se realizarán.")


NaNs en X: False
Infs en X: False
NaNs en y_scaled: False
Infs en y_scaled: False
Forma de X_train: (1042, 28)
Forma de X_test: (261, 28)
Forma de y_train: (1042, 1)
Forma de y_test: (261, 1)

Iniciando entrenamiento de la Red Neuronal...
Epoch 0 | Loss: 0.2132
Epoch 100 | Loss: 0.1233
Epoch 200 | Loss: 0.0778
Epoch 300 | Loss: 0.0540
Epoch 400 | Loss: 0.0409
Epoch 500 | Loss: 0.0333
Epoch 600 | Loss: 0.0286
Epoch 700 | Loss: 0.0256
Epoch 800 | Loss: 0.0235
Epoch 900 | Loss: 0.0219

Realizando predicciones en el conjunto de prueba...

Métricas de Evaluación en el Conjunto de Prueba:
Error Cuadrático Medio (MSE): 19369.7462
Raíz del Error Cuadrático Medio (RMSE): 139.1752
Coeficiente de Determinación (R²): 0.7463


## Comparación de arquitecturas y funciones de activación


Experimentar con diferentes arquitecturas de red (número de capas ocultas y neuronas por capa). Comparar el rendimiento utilizando diferentes funciones de activación (Sigmoid, ReLU, Tanh) en las capas ocultas. Registrar y comparar las métricas de evaluación para cada configuración.


In [46]:
architectures = [
    [X_train.shape[1], 64, 32, 1],
    [X_train.shape[1], 128, 64, 1],
    [X_train.shape[1], 64, 64, 1],
    [X_train.shape[1], 128, 64, 32, 1]
]


activation_functions = ['relu', 'sigmoid', 'tanh']


results = []

epochs = 1000
learning_rate = 0.001

print("Experimentando con diferentes estructuras...")

for arch in architectures:
    for activation in activation_functions:
        print(f"\nExperimentando con: {arch}, Activation: {activation}")


        nn = NeuralNetwork(layers=arch, activation=activation)
        nn.train(X_train, y_train, epochs=epochs, learning_rate=learning_rate)


        predictions_scaled = nn.predict(X_test)


        predictions_original = scaler_Y.inverse_transform(predictions_scaled)
        y_test_original = scaler_Y.inverse_transform(y_test)


        mse = mean_squared_error(y_test_original, predictions_original)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test_original, predictions_original)

        results.append({
            'Architecture': arch,
            'Activation': activation,
            'MSE': mse,
            'RMSE': rmse,
            'R2': r2
        })


        print(f"  MSE: {mse:.4f}")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  R2: {r2:.4f}")

print("\nExperimentacion completa")


results_df = pd.DataFrame(results)

results_df_sorted = results_df.sort_values(by='R2', ascending=False)

print("\nResultados:")
display(results_df_sorted)


Starting experimentation with different architectures and activation functions...

Experimenting with Architecture: [28, 64, 32, 1], Activation: relu
Epoch 0 | Loss: 0.4157
Epoch 100 | Loss: 0.2175
Epoch 200 | Loss: 0.1591
Epoch 300 | Loss: 0.1324
Epoch 400 | Loss: 0.1161
Epoch 500 | Loss: 0.1047
Epoch 600 | Loss: 0.0958
Epoch 700 | Loss: 0.0886
Epoch 800 | Loss: 0.0824
Epoch 900 | Loss: 0.0771
  MSE: 65876.7131
  RMSE: 256.6646
  R2: 0.1372

Experimenting with Architecture: [28, 64, 32, 1], Activation: sigmoid
Epoch 0 | Loss: 0.5895
Epoch 100 | Loss: 0.1518
Epoch 200 | Loss: 0.0937
Epoch 300 | Loss: 0.0858
Epoch 400 | Loss: 0.0845
Epoch 500 | Loss: 0.0841
Epoch 600 | Loss: 0.0839
Epoch 700 | Loss: 0.0836
Epoch 800 | Loss: 0.0833
Epoch 900 | Loss: 0.0831
  MSE: 67026.9550
  RMSE: 258.8956
  R2: 0.1221

Experimenting with Architecture: [28, 64, 32, 1], Activation: tanh
Epoch 0 | Loss: 0.2877
Epoch 100 | Loss: 0.1643
Epoch 200 | Loss: 0.1013
Epoch 300 | Loss: 0.0681
Epoch 400 | Loss: 0.0

,Architecture,Activation,MSE,RMSE,R2
11,"[28, 128, 64, 32, 1]",tanh,18809.884195,137.149131,0.753631
5,"[28, 128, 64, 1]",tanh,21063.915434,145.134129,0.724108
2,"[28, 64, 32, 1]",tanh,21632.116572,147.078607,0.716666
6,"[28, 64, 64, 1]",relu,28957.595421,170.169314,0.620718
8,"[28, 64, 64, 1]",tanh,30431.353542,174.445847,0.601415
3,"[28, 128, 64, 1]",relu,40194.440532,200.485512,0.473540
9,"[28, 128, 64, 32, 1]",relu,43505.285135,208.579206,0.430175
0,"[28, 64, 32, 1]",relu,65876.713103,256.664593,0.137158
1,"[28, 64, 32, 1]",sigmoid,67026.955028,258.895645,0.122092
7,"[28, 64, 64, 1]",sigmoid,73531.190674,271.166352,0.036901


In [47]:

file_path = '/content/laptop_price.csv'

try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(file_path, encoding='latin1')
except FileNotFoundError:
    print(f"Error: El archivo no se encontró en la ruta especificada: {file_path}")
    # Attempt to mount drive again just in case
    from google.colab import drive
    drive.mount('/content/drive')
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='latin1')
    except FileNotFoundError:
        print("Error: El archivo aún no se encontró después de intentar montar el Drive.")
        df = None # Set df to None to indicate failure

if df is not None:
    # Convertir Ram de "8GB" → 8
    df['Ram'] = df['Ram'].str.replace('GB', '').astype(int)

    # Convertir Weight de "2.1kg" → 2.1
    df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)


    np.random.seed(42)
    base = 1000 - df['Price_euros'] * 0.7 + df['Ram'] * 5
    ruido = np.random.normal(0, 30, size=len(df))
    df['Demanda'] = np.clip(base + ruido, 0, None)


    # 1. Seleccionamos las columnas de entrada
    features = ['Company', 'TypeName', 'Ram', 'Weight', 'Price_euros']
    X_raw = df[features]

    # 2. One-hot encoding para las categóricas
    categorical_cols = ['Company', 'TypeName']
    numerical_cols = ['Ram', 'Weight', 'Price_euros']

    # One-hot encoding (devuelve un array)
    encoder = OneHotEncoder(sparse_output=False)
    X_cat = encoder.fit_transform(X_raw[categorical_cols])

    # Escalamiento de las numéricas (media = 0, varianza = 1)
    scaler = StandardScaler()
    X_num = scaler.fit_transform(X_raw[numerical_cols])

    # 3. Concatenamos todo
    X = np.hstack([X_cat, X_num])

    # 4. Salida: y = Demanda
    y = df['Demanda'].values.reshape(-1, 1)

    # 5. Escalar la variable objetivo y
    scaler_Y = MinMaxScaler()
    y_scaled = scaler_Y.fit_transform(y)

    print("NaNs en X:", np.any(np.isnan(X)))
    print("Infs en X:", np.any(np.isinf(X)))
    print("NaNs en y_scaled:", np.any(np.isnan(y_scaled)))
    print("Infs en y_scaled:", np.any(np.isinf(y_scaled)))

    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error, r2_score

    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=42)

    # Verificar las formas de los conjuntos resultantes
    print("Forma de X_train:", X_train.shape)
    print("Forma de X_test:", X_test.shape)
    print("Forma de y_train:", y_train.shape)
    print("Forma de y_test:", y_test.shape)


    architectures = [
        [X_train.shape[1], 64, 32, 1],
        [X_train.shape[1], 128, 64, 1],
        [X_train.shape[1], 64, 64, 1],
        [X_train.shape[1], 128, 64, 32, 1]
    ]

    activation_functions = ['relu', 'sigmoid', 'tanh']

    results = []

    epochs = 1000
    learning_rate = 0.001

    print("\nExperimentacion...")

    for arch in architectures:
        for activation in activation_functions:
            print(f"\nExperimentando con : {arch}, Activation: {activation}")

            nn = NeuralNetwork(layers=arch, activation=activation)
            nn.train(X_train, y_train, epochs=epochs, learning_rate=learning_rate)

            predictions_scaled = nn.predict(X_test)


            predictions_original = scaler_Y.inverse_transform(predictions_scaled)
            y_test_original = scaler_Y.inverse_transform(y_test)

            mse = mean_squared_error(y_test_original, predictions_original)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test_original, predictions_original)

            results.append({
                'Architecture': arch,
                'Activation': activation,
                'MSE': mse,
                'RMSE': rmse,
                'R2': r2
            })

            print(f"  MSE: {mse:.4f}")
            print(f"  RMSE: {rmse:.4f}")
            print(f"  R2: {r2:.4f}")

    print("\nExperimentacion completa")


    results_df = pd.DataFrame(results)

    results_df_sorted = results_df.sort_values(by='R2', ascending=False)

    print("\nResultados:")
    display(results_df_sorted)

else:
    print("error carga de datos")


NaNs en X: False
Infs en X: False
NaNs en y_scaled: False
Infs en y_scaled: False
Forma de X_train: (1042, 28)
Forma de X_test: (261, 28)
Forma de y_train: (1042, 1)
Forma de y_test: (261, 1)

Starting experimentation with different architectures and activation functions...

Experimenting with Architecture: [28, 64, 32, 1], Activation: relu
Epoch 0 | Loss: 0.1486
Epoch 100 | Loss: 0.0942
Epoch 200 | Loss: 0.0716
Epoch 300 | Loss: 0.0588
Epoch 400 | Loss: 0.0508
Epoch 500 | Loss: 0.0453
Epoch 600 | Loss: 0.0413
Epoch 700 | Loss: 0.0383
Epoch 800 | Loss: 0.0359
Epoch 900 | Loss: 0.0340
  MSE: 32072.4755
  RMSE: 179.0879
  R2: 0.5799

Experimenting with Architecture: [28, 64, 32, 1], Activation: sigmoid
Epoch 0 | Loss: 0.2918
Epoch 100 | Loss: 0.1190
Epoch 200 | Loss: 0.0988
Epoch 300 | Loss: 0.0961
Epoch 400 | Loss: 0.0955
Epoch 500 | Loss: 0.0952
Epoch 600 | Loss: 0.0948
Epoch 700 | Loss: 0.0945
Epoch 800 | Loss: 0.0941
Epoch 900 | Loss: 0.0938
  MSE: 75712.8097
  RMSE: 275.1596
  R2: 0

,Architecture,Activation,MSE,RMSE,R2
11,"[28, 128, 64, 32, 1]",tanh,14833.586004,121.793210,0.805712
5,"[28, 128, 64, 1]",tanh,25827.915251,160.710657,0.661710
8,"[28, 64, 64, 1]",tanh,27596.659014,166.122422,0.638544
2,"[28, 64, 32, 1]",tanh,31028.623872,176.149436,0.593592
0,"[28, 64, 32, 1]",relu,32072.475481,179.087899,0.579920
9,"[28, 128, 64, 32, 1]",relu,33590.959548,183.278366,0.560031
3,"[28, 128, 64, 1]",relu,37217.735783,192.918988,0.512528
6,"[28, 64, 64, 1]",relu,44941.293248,211.993616,0.411367
7,"[28, 64, 64, 1]",sigmoid,67750.031877,260.288363,0.112622
4,"[28, 128, 64, 1]",sigmoid,69259.098545,263.171234,0.092856


## Conclusiones


Analizar los resultados obtenidos con las diferentes configuraciones (arquitecturas y funciones de activación). Identificar la arquitectura y función de activación que mejor rendimiento tuvieron para este problema.


In [49]:

display(results_df_sorted)


best_config = results_df_sorted.iloc[0]

print("\nMejor Configuración Encontrada:")
print(f"  Arquitectura: {best_config['Architecture']}")
print(f"  Función de Activación: {best_config['Activation']}")
print(f"  R2 Score: {best_config['R2']:.4f}")
print(f"  MSE: {best_config['MSE']:.4f}")
print(f"  RMSE: {best_config['RMSE']:.4f}")


print("\nAnálisis de Resultados:")
print("Se compararon diferentes arquitecturas de red neuronal y funciones de activación (ReLU, Sigmoid, Tanh).")
print(f"La mejor combinación encontrada fue la arquitectura {best_config['Architecture']} con la función de activación '{best_config['Activation']}',")
print(f"obteniendo el R² más alto ({best_config['R2']:.4f}) en el conjunto de prueba.")


,Architecture,Activation,MSE,RMSE,R2
11,"[28, 128, 64, 32, 1]",tanh,14833.586004,121.793210,0.805712
5,"[28, 128, 64, 1]",tanh,25827.915251,160.710657,0.661710
8,"[28, 64, 64, 1]",tanh,27596.659014,166.122422,0.638544
2,"[28, 64, 32, 1]",tanh,31028.623872,176.149436,0.593592
0,"[28, 64, 32, 1]",relu,32072.475481,179.087899,0.579920
9,"[28, 128, 64, 32, 1]",relu,33590.959548,183.278366,0.560031
3,"[28, 128, 64, 1]",relu,37217.735783,192.918988,0.512528
6,"[28, 64, 64, 1]",relu,44941.293248,211.993616,0.411367
7,"[28, 64, 64, 1]",sigmoid,67750.031877,260.288363,0.112622
4,"[28, 128, 64, 1]",sigmoid,69259.098545,263.171234,0.092856



Mejor Configuración Encontrada:
  Arquitectura: [28, 128, 64, 32, 1]
  Función de Activación: tanh
  R2 Score: 0.8057
  MSE: 14833.5860
  RMSE: 121.7932

Análisis de Resultados:
Se compararon diferentes arquitecturas de red neuronal y funciones de activación (ReLU, Sigmoid, Tanh).
La mejor combinación encontrada fue la arquitectura [28, 128, 64, 32, 1] con la función de activación 'tanh',
obteniendo el R² más alto (0.8057) en el conjunto de prueba.
